In [1]:
!pip install -q pydub
!pip install -q tinkoff-voicekit-client protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 108.6 MB/s eta 0:0

In [2]:
from IPython.display import Audio
from pydub import AudioSegment
import getpass
from tinkoff_voicekit_client import ClientSTT
from pprint import pprint

In [3]:
Audio('/content/call.mp3')

In [4]:
# чтение из файла
call = AudioSegment.from_file(file='/content/call.mp3',
                               format='mp3')

print(f"Продолжительность аудио: {call.duration_seconds} секунд")

print(f"Частота дискретизаци: {call.frame_rate}")

print(f"Количество каналов: {call.channels}")

# представление 15 секунд в миллисекундах
FORTY_SECONDS = 15 * 1000

# обрезка файла начиная с 15 сек при помощи индексации
call[FORTY_SECONDS::]

Продолжительность аудио: 90.0 секунд
Частота дискретизаци: 16000
Количество каналов: 2


In [5]:
# сохраним фрагмент в файл
call[15000::].export('call_trim.mp3', format='mp3')

<_io.BufferedRandom name='call_trim.mp3'>

In [6]:
# передаем API_KEY
API_KEY = getpass.getpass("Tinkoff API Key:")

Tinkoff API Key:··········


In [7]:
# передаем SECRET_KEY
SECRET_KEY = getpass.getpass("Tinkoff SECRET Key:")

Tinkoff SECRET Key:··········


In [9]:
# Создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# Указываем параметры аудио
audio_config = {
    "encoding": "MPEG_AUDIO",             # кодировка
    "sample_rate_hertz": 16000,           # частота
    "num_channels": 2,                    # количество каналов
    "enable_automatic_punctuation": True  # включаем пунктуацию
}

# Указываем параметры потока
stream_config = {"config": audio_config}

# Пустой список для результатов
combined_results = []

# Вызываем метод streaming_recognize и обрабатываем каждый ответ
with open("/content/call.mp3", "rb") as source:
    responses = client.streaming_recognize(source, stream_config)
    for response in responses:
        pprint(response)

        # Добавляем результаты в список
        combined_results.extend(response.get('results', []))

{'results': [{'is_final': True,
              'recognition_result': {'alternatives': [{'confidence': -3.6970563,
                                                       'transcript': 'Алло.',
                                                       'words': [{'confidence': 0.0,
                                                                  'end_time': '18.240s',
                                                                  'start_time': '18.030s',
                                                                  'word': 'алло'}]}],
                                     'channel': 1,
                                     'end_time': '18.240s',
                                     'start_time': '18.030s'},
              'stability': 0.0}]}
{'results': [{'is_final': True,
              'recognition_result': {'alternatives': [{'confidence': -4.7951875,
                                                       'transcript': 'Здравствуйте, '
                                                   

In [10]:
# Функция для создания диалога
def create_dialog(results):
    dialog_text = ""

    for entry in results:
        recognition_result = entry['recognition_result']
        speaker = "Менеджер" if recognition_result['channel'] == 0 else "Клиент"
        transcript = recognition_result['alternatives'][0]['transcript']

        dialog_text += f"{speaker}: {transcript}\n"

    return dialog_text

# Создание диалога
dialog_text = create_dialog(combined_results)
print(dialog_text)

Клиент: Алло.
Менеджер: Здравствуйте, Алина.
Клиент: Да.
Менеджер: Меня зовут Юлия, у меня тет искусственного интеллекта. От вас пришла заявка на бесплатный вебинар, все верно.
Клиент: Наверное, да, не помню уже.
Менеджер: Мы хотим говорить дополнительные материалы к бесплатному вебинару, могу задать несколько вопросов.
Клиент: Да, давайте.
Менеджер: Скажите, планируете изучать нейронные сети для себя, для ребенка или для сотрудника?
Клиент: Так до свидания, поняла, о чем ага.
Менеджер: Так для себя скажите, а вы программист или нет?
Клиент: Нет пока еще, ну, в смысле Чече.
Менеджер: А какая у вас?
Менеджер: Что-что.
Клиент: Нет, еще нет.
Менеджер: А какая у вас сейчас профессия?
Клиент: Ну, я изучаю мен-ну, то есть не связано с Туганова.
Менеджер: А что, что, плохо слышно, что вы сказали?
Клиент: Программирование не связано.
Менеджер: А какая правивая сейчас у вас?
Клиент: Извините, договорились.
Клиент: Ага.
Менеджер: Хорошо, ладно, вы хотите применять нейронные сети в своей процессе

In [11]:
def create_continuous_transcript(results):
    transcript_text = ""

    for entry in results:
        recognition_result = entry['recognition_result']
        transcript = recognition_result['alternatives'][0]['transcript']
        transcript_text += f"{transcript} "

    return transcript_text.strip()

# Создание непрерывной строки
continuous_transcript = create_continuous_transcript(combined_results)
print(continuous_transcript)


Алло. Здравствуйте, Алина. Да. Меня зовут Юлия, у меня тет искусственного интеллекта. От вас пришла заявка на бесплатный вебинар, все верно. Наверное, да, не помню уже. Мы хотим говорить дополнительные материалы к бесплатному вебинару, могу задать несколько вопросов. Да, давайте. Скажите, планируете изучать нейронные сети для себя, для ребенка или для сотрудника? Так до свидания, поняла, о чем ага. Так для себя скажите, а вы программист или нет? Нет пока еще, ну, в смысле Чече. А какая у вас? Что-что. Нет, еще нет. А какая у вас сейчас профессия? Ну, я изучаю мен-ну, то есть не связано с Туганова. А что, что, плохо слышно, что вы сказали? Программирование не связано. А какая правивая сейчас у вас? Извините, договорились. Ага. Хорошо, ладно, вы хотите применять нейронные сети в своей процессе, в своей работе. Ну, скоре-то лечение, наверное, так, то есть да. УГУ, хорошо, а вы по собственной инициативе, инициативе компании где работаете? Посадки на. Возможно, у вас есть какой-то проект, к